# CADEvolve-seminar: "подстроиться под заказчика" без данных для обучения

Этот ноутбук — **общий “гайд по пайплайну”** для ситуации, когда:

- **нет** готового датасета/разметки под нового заказчика,
- есть **описание** того, какие детали нужны (семейства деталей + параметры),
- и нужно быстро собрать **синтетический train-set** и обучить модель под этот стиль деталей.

Идея: мы **генерируем данные сами**, стартуя от текстового брифа заказчика и LLM-помощника, затем прогоняем стандартную подготовку датасета и обучение.




## Шаг 0 — создать окружение

В корне репозитория:


In [ ]:
!conda env create -f ./environment.yml
# затем:
# conda activate cadevolve-seminar

---

## Шаг 1 — прочитать бриф заказчика

Каждая команда получает **свой** бриф (требования заказчика) в `./clients_briefs/`.

1) Найдите файл с номером вашей команды. Там вы найдете описание семейства деталей, восстановление которого требует заказчик 
2) Выпишите для себя:
- название класса деталей
- краткое геометрическое описание,
- развернутое геометрическое описание,

Подсказка: вам не нужно идеально воспроизводить каждый элемент. Цель — **восстановление принципиальной формы**.


In [ ]:
from pathlib import Path

brief_path = Path("./clients_briefs/client_0*")
print(brief_path.read_text(encoding="utf-8"))

---

## Шаг 2 — LLM-based bootstrap генератора (CadQuery)

Дальше вы работаете в ноутбуке:

`./llm_based generation/CADEvolve_seminar_generator_bootstrap.ipynb`

Там вы:

- выбираете похожие семейства деталей (similarity search),
- собираете few-shot примеры,
- **синтезируете** новый CadQuery-генератор для семейства заказчика,
- при необходимости делаете **repair loop** (правки по ошибкам/валидаторам).

### Итог Шага 2
Файл генератора должен оказаться здесь:

`./data/generators/client_part.py`

И он должен содержать **ровно одну** функцию-генератор с параметрами по умолчанию, например:

```python
def my_customer_part(a: float = 10.0, b: float = 2.0, ...) -> "cq.Workplane":
    import cadquery as cq
    ...
    return body
```




In [ ]:
from pathlib import Path

gen_path = Path("./data/generators/client_part.py")
print("Generator path:", gen_path)
print("Exists:", gen_path.exists())

---

## Шаг 3 — сэмплирование данных + нормализация + бинаризация + рот. аугментация

Когда генератор готов и лежит в `./data/generators/client_part.py`, вы генерируете синтетические данные.

### 3.1 Сгенерировать данные (bash-пайплайн)
Команды (в корне репо):

```bash
cd ./dataset_utils
bash ./generate_data.sh
```

Этот шаг обычно делает:
- семплирование параметров и генерацию STL,
- стандартизацию масштаба/позиции/ориентации,
- бинаризацию,
- ротационную аугментацию.


### 3.2 Собрать train-set в pickle

После того как `generate_data.sh` отработал, создаём `.pkl` с парами `(py_path, stl_path)`:

```bash
python ./utils/create_pkl.py \
  --py_dir  ../data/results/rotated \
  --stl_dir ../data/results/rotated_stl \
  --out_dir ../data/train_set
```

В итоге вы получаете pickle-файл с train-set.


In [ ]:
# Выполните команды ниже из терминала (или прямо из ноутбука).
# ВНИМАНИЕ: это может занять время и много диска.

# %cd ./dataset_utils
# !bash ./generate_data.sh
# !python ./utils/create_pkl.py --py_dir ../data/results/rotated --stl_dir ../data/results/rotated_stl --out_dir ../data/train_set

Проверка: убедитесь, что pkl появился и в нём есть записи.

In [ ]:
import pickle
from pathlib import Path

pkl_path = Path("/Users/zhemchuzhnikov/CADEvolve seminar/data/train_set/pairs.pkl")

with open(pkl_path, "rb") as f:
    items = pickle.load(f)

print("Loaded:", pkl_path)
print("Items:", len(items))
print("Type(items):", type(items))
print("Example item:", items[0] if items else None)

---

## Шаг 4 — обучение

Переходим в `./train` и запускаем обучение:

```bash
cd ./train
python ./train.py --config ./config.yaml
```

Важно:
- `config.yaml` должен указывать на ваш `items_pkl` (из Шага 3),
- `model.model_id` может быть базовой моделью или вашей контрольной точкой на HF,
- `dataloader_num_workers` в headless окружениях иногда лучше ставить небольшим (0–2),
- если рендер падает из-за X server, обычно решается заменой рендер-стека на headless (vtk-osmesa / offscreen) или подготовкой данных заранее.

> На семинаре мы ожидаем, что train-скрипт просто стартует и сделает хотя бы короткий прогон.


In [ ]:
# Терминальные команды (пример):
# %cd ./train
# !python ./train.py --config ./config.yaml

---

## Шаг 5 — тестирование на примерах семинара

В конце семинара в репозиторий добавятся тестовые примеры.
Вам нужно будет проверить вашу модель через:

`./test.ipynb`

Там обычно:
- загружается ваша модель (из output_dir или с HuggingFace),
- прогоняется инференс на тестовых примерах,
- делается экспорт STL и базовые проверки (не пусто, bounds, watertight и т.п.).

> Если тесты падают: чаще всего проблема в несовместимости версий или в том, что модель/процессор загружены не из одного и того же репозитория/папки.
